<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-of-Python-modules" data-toc-modified-id="Import-of-Python-modules-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import of Python modules</a></span></li><li><span><a href="#Class-for-generation-of-blade-profile" data-toc-modified-id="Class-for-generation-of-blade-profile-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Class for generation of blade profile</a></span></li><li><span><a href="#Optimization-loop" data-toc-modified-id="Optimization-loop-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Optimization loop</a></span></li></ul></div>

## Import of Python modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from scipy import interpolate
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize, differential_evolution, shgo, basinhopping
import pyDOE

In [2]:
path = "C:/Users/Persik/Anaconda3/envs/py36/Library/bin/"
sys.path.append(path)

import FreeCAD, Part

## Class for generation of blade profile

In [3]:
class BladeProfile():
    def __init__(self, L, Beta_array, R_array, T_array, CoefLe=1, CoefTe=1, N=5000, N_LeTe = 30):
        self.L = L
        self.Beta_array = Beta_array
        self.R_array = R_array
        self.T_array = T_array
        self.CoefLe = CoefLe
        self.CoefTe = CoefTe
        self.N = N
        self.N_LeTe = N_LeTe
    
    def Generate2Dprofile(self):
        '''Generation 2D profile of the blade, function returns XY coordinates of the profile'''
        L = self.L
        Beta_array = self.Beta_array
        R_array = self.R_array
        T_array = self.T_array
        CoefLe = self.CoefLe
        CoefTe = self.CoefTe
        N = self.N
        N_LeTe = self.N_LeTe
        
        FirstP = CoefLe*T_array[0]*np.cos(Beta_array[0]/180*np.pi)/2.0
        LastP = CoefTe*T_array[-1]*np.cos(Beta_array[-1]/180*np.pi)/2.0
        dx_i = np.linspace(0+FirstP, L-LastP, N)    # Split of the horizontal length of the blade
        dx = dx_i[1:]-dx_i[:-1]
        dx_beta_func = np.linspace(0, L, Beta_array.shape[0])   # Split length of blade for generation function beta = f(x)
        beta_func = interpolate.interp1d(dx_beta_func, Beta_array, kind='quadratic') # function beta f(x)
        Beta_i = beta_func(dx_i[1:])       # Calculation of the beta values for dx coordinates    
        dy = dx*np.tan(Beta_i/180.0*np.pi)    # Calculation dy coordinates of the blade
        dy_i = np.array([dy[:i+1].sum() for i in range(dy.shape[0])])+np.sqrt((CoefLe*T_array[0])**2/4-FirstP**2)

        # Thickness of the streamline
        dx_thick_func = np.linspace(0, L, T_array.shape[0])
        thick_func = interpolate.interp1d(dx_thick_func, T_array, kind='quadratic')
        T_i = thick_func(dx_i[1:])

        dl = np.sqrt(dx**2+dy**2)
        Alpha_i = np.arctan(T_i/dl/2)
        dm = np.sqrt(dl**2+(T_i/2.0)**2)
        T_y1 = dm*np.sin(Alpha_i+(Beta_i/180*np.pi))
        T_x1 = dm*np.cos(Alpha_i+(Beta_i/180*np.pi))
        T_y2 = dm*np.sin((Beta_i/180*np.pi)-Alpha_i)
        T_x2 = dm*np.cos((Beta_i/180*np.pi)-Alpha_i)

        T_y1i = T_y1+dy_i
        T_x1i = T_x1+dx_i[1:]
        T_y2i = T_y2+dy_i
        T_x2i = T_x2+dx_i[1:]
        
        if CoefLe == 1:
            # Creation of the coordinates of Leading Edge of the Airfoil
            ArcLe = Part.Arc(FreeCAD.Vector(T_x2i[0], T_y2i[0], 0), FreeCAD.Vector(0, 0, 0), FreeCAD.Vector(T_x1i[0], T_y1i[0], 0))
            Le_list = ArcLe.discretize(Number=N_LeTe+2)
            Le_x = np.array([Le_list[i].x for i in range(len(Le_list))])
            Le_y = np.array([Le_list[i].y for i in range(len(Le_list))])
        else:   
            # Ellipse type of the Leading Edge
            MajorR = App.Vector(0, 0, 0)
            MinorR = App.Vector(T_x1i[0], T_y1i[0], 0)
            Center = App.Vector(dx_i[0], dy_i[0], 0)
            ArcEllLe = Part.ArcOfEllipse(Part.Ellipse(MajorR, MinorR, Center),-1.5, 1.5)
            Le_list = ArcEllLe.discretize(Number=N_LeTe)
            Le_list.append(MinorR)
            Le_list.insert(0, App.Vector(T_x2i[0], T_y2i[0], 0))
            Le_x = np.array([Le_list[i].x for i in range(len(Le_list))])
            Le_y = np.array([Le_list[i].y for i in range(len(Le_list))])    
        
        if CoefTe == 1:
            #Creation of the coordinates Trailing Edge of the Airfoil
            LastP_y = dy_i[-1]+np.sqrt((CoefTe*T_array[-1])**2/4-LastP**2)
            ArcTe = Part.Arc(FreeCAD.Vector(T_x1i[-1], T_y1i[-1], 0), FreeCAD.Vector(L, LastP_y, 0), FreeCAD.Vector(T_x2i[-1], T_y2i[-1], 0))
            Te_list = ArcTe.discretize(Number=N_LeTe+2)
            Te_x = np.array([Te_list[i].x for i in range(len(Te_list))])
            Te_y = np.array([Te_list[i].y for i in range(len(Te_list))])
        else:
            # Ellipse type of the Trailling Edge
            LastP_y = dy_i[-1]+np.sqrt((CoefTe*T_array[-1])**2/4-LastP**2)
            MajorR_Te = App.Vector(L, LastP_y, 0)
            MinorR_Te = App.Vector(T_x1i[-1], T_y1i[-1], 0)
            Center_Te = App.Vector(dx_i[-1], dy_i[-1], 0)
            ArcEllTe = Part.ArcOfEllipse(Part.Ellipse(MajorR_Te, MinorR_Te, Center_Te),-1.5, 1.5)
            Te_list = ArcEllTe.discretize(Number=N_LeTe)
            Te_list.append(MinorR_Te)
            Te_list.insert(0, App.Vector(T_x2i[-1], T_y2i[-1], 0))
            Te_x = np.flip(np.array([Te_list[i].x for i in range(len(Te_list))]))
            Te_y = np.flip(np.array([Te_list[i].y for i in range(len(Te_list))]))      
        
        # Compound of the all XY coordinates
        X1 = np.array([Le_x, T_x1i, Te_x, np.flip(T_x2i)])
        Y1 = np.array([Le_y, T_y1i, Te_y, np.flip(T_y2i)])
        
        # Creation of the list with FreeCAD Vectors of the streamline
        ListStreamline = []
        for i in range(dy_i.shape[0]):
            v_i = FreeCAD.Vector(dx_i[i], dy_i[i], 0)
            ListStreamline.append(v_i)
        
        SplineStreamline = Part.BSplineCurve()
        SplineStreamline.interpolate(ListStreamline)
        SplineStreamline = SplineStreamline.toShape()
        CenterMass = SplineStreamline.CenterOfMass
        
        X = X1-CenterMass.x
        Y = Y1-CenterMass.y
        
        return X, Y
    
    def View2DAirfoil(self, X_arrays, Y_arrays):
        X1 = np.append(X_arrays[0], X_arrays[1])
        Y1 = np.append(Y_arrays[0], Y_arrays[1])
        X2 = np.append(X_arrays[2], X_arrays[3])
        Y2 = np.append(Y_arrays[2], Y_arrays[3])
        X = np.append(X1, X2)
        Y = np.append(Y1, Y2)
        plt.xlabel('X, [mm]')
        plt.ylabel('Y, [mm]')
        plt.plot(X, Y)
        plt.grid()
        plt.axis('equal')
        plt.show()
    
    def Generate3Dprofile(self, X_arrays, Y_arrays):
        '''Generation of the 3D coordinates of the profile with using Conformal Method'''
        L = self.L
        Beta_array = self.Beta_array
        R_array = self.R_array
        T_array = self.T_array
        CoefLe = self.CoefLe
        CoefTe = self.CoefTe
        N = self.N
        N_LeTe = self.N_LeTe
          
        merid_func = interpolate.interp1d(R_array[0], R_array[1], kind='quadratic')
        R_i1 = merid_func(Y_arrays[1])
        x_ps = R_i1*np.cos(X_arrays[1]/R_i1)
        y_ps = R_i1*np.sin(X_arrays[1]/R_i1)
        z_ps = Y_arrays[1]

        R_i2 = merid_func(Y_arrays[3])
        x_ss = R_i2*np.cos(X_arrays[3]/R_i2)
        y_ss = R_i2*np.sin(X_arrays[3]/R_i2)
        z_ss = Y_arrays[3]

        R_le = merid_func(Y_arrays[0])
        x_le = R_le*np.cos(X_arrays[0]/R_le)
        y_le = R_le*np.sin(X_arrays[0]/R_le)
        z_le = Y_arrays[0]

        R_te = merid_func(Y_arrays[2])
        x_te = R_te*np.cos(X_arrays[2]/R_te)
        y_te = R_te*np.sin(X_arrays[2]/R_te)
        z_te = Y_arrays[2]

        x1 = np.append(x_le, x_ps)
        y1 = np.append(y_le, y_ps)
        z1 = np.append(z_le, z_ps)
        x2 = np.append(x_te, x_ss)
        y2 = np.append(y_te, y_ss)
        z2 = np.append(z_te, z_ss)
        x = np.append(x1, x2)
        y = np.append(y1, y2)
        z = np.append(z1, z2)
        
        return x, y, z

    def View3DAirfoil(self, X, Y, Z):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot(X, Y, Z)
        ax.set_aspect('equal')
        
        max_range = np.array([X.max()-X.min(), Y.max()-Y.min(), Z.max()-Z.min()]).max()
        Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(X.max()+X.min())
        Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(Y.max()+Y.min())
        Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(Z.max()+Z.min())
        # Comment or uncomment following both lines to test the fake bounding box:
        for xb, yb, zb in zip(Xb, Yb, Zb):
           ax.plot([xb], [yb], [zb], 'w')
        
    def ProfileToFreeCAD(self, X, Y, Z):
        N_LeTe = self.N_LeTe
        N = self.N
        
        le_v = [FreeCAD.Vector(X[i], Y[i], Z[i]) for i in range(N_LeTe+2)]
        ps_v = [FreeCAD.Vector(X[i+N_LeTe+2], Y[i+N_LeTe+2], Z[i+N_LeTe+2]) for i in range(N-1)]
        te_v = [FreeCAD.Vector(X[i+(N+N_LeTe+1)], Y[i+(N+N_LeTe+1)], Z[i+(N+N_LeTe+1)]) for i in range(N_LeTe+2)]
        ss_v = [FreeCAD.Vector(X[i+(N+2*N_LeTe+3)], Y[i+(N+2*N_LeTe+3)], Z[i+(N+2*N_LeTe+3)]) for i in range(N-1)]
        
        LeSpline = Part.BSplineCurve()
        LeSpline.interpolate(le_v)
        LeSpline = LeSpline.toShape()
        
        PsSpline = Part.BSplineCurve()
        ps_arr = np.array(ps_v[:-1])
        ps_v = ps_arr[::50].tolist()
        ps_v.append(te_v[0])
        PsSpline.interpolate(ps_v)
        PsSpline = PsSpline.toShape()

        TeSpline = Part.BSplineCurve()
        TeSpline.interpolate(te_v)
        TeSpline = TeSpline.toShape()
        
        SsSpline = Part.BSplineCurve()
        ss_arr = np.array(ss_v[:-1])
        ss_v = ss_arr[::50].tolist()
        ss_v.append(le_v[0])
        SsSpline.interpolate(ss_v)
        SsSpline = SsSpline.toShape()
        
        W = Part.Wire([LeSpline, PsSpline, TeSpline, SsSpline])

        return W

## Optimization loop

In [4]:
FreeCAD.newDocument()
Part.open('./ReferenceBlade.step')
ObjBlade = FreeCAD.ActiveDocument.getObject('ReferenceBlade')
SolidBladeRef = ObjBlade.Shape
CenterMassRefBlade = SolidBladeRef.CenterOfMass
SolidBladeRef2 = SolidBladeRef.copy()
SolidBladeRef2.translate(FreeCAD.Vector(0, 0, -CenterMassRefBlade.z))
Part.show(SolidBladeRef2)
FreeCAD.ActiveDocument.removeObject('ReferenceBlade')

# Read Shroud and Hub coordinates from .csv files with XYZ
df_hub = pd.read_csv('./Rotor37/hub.curve', decimal='.', delimiter='\s+', names=['X', 'Y', 'Z'], header=None)*10.0
df_shroud = pd.read_csv('./Rotor37/shroud.curve', decimal='.', delimiter='\s+', names=['X', 'Y', 'Z'], header=None)*10.0
df_shroud['Z'] = df_shroud['Z']-CenterMassRefBlade.z
df_hub['Z'] = df_hub['Z']-CenterMassRefBlade.z
df_ave = pd.DataFrame()
df_ave['X'] = (df_shroud['X']+df_hub['X'])/2.0
df_ave['Z'] = (df_shroud['Z']+df_hub['Z'])/2.0

# Generate Ave lines of the reference blade
AveMerVectors = []
for i in range(df_ave.shape[0]):
    v_i = FreeCAD.Vector(0, df_ave['X'][i], df_ave['Z'][i])
    AveMerVectors.append(v_i)
    
AveSpline = Part.BSplineCurve()
AveSpline.interpolate(AveMerVectors)
AveSpline = AveSpline.toShape()
AveSurface = AveSpline.revolve(App.Vector(0, 0, 0), App.Vector(0, 0, 1), -180)

# Lines for
BladeRefAvePsLine = SolidBladeRef2.Faces[1].Surface.intersectSS(AveSurface.Surface)[0].toShape()
BladeRefAvePsList = BladeRefAvePsLine.discretize(Number=10)

BladeRefAveSsLine = SolidBladeRef2.Faces[3].Surface.intersectSS(AveSurface.Surface)[0].toShape()
BladeRefAveSsList = BladeRefAveSsLine.discretize(Number=10)

BladeRefHubPsLine = SolidBladeRef2.Faces[1].Edges[1]
BladeRefHubPsList = BladeRefHubPsLine.discretize(Number=10)

BladeRefHubSsLine = SolidBladeRef2.Faces[3].Edges[1]
BladeRefHubSsList = BladeRefHubSsLine.discretize(Number=10)

BladeRefShroudPsLine = SolidBladeRef2.Faces[1].Edges[3]
BladeRefShroudPsList = BladeRefShroudPsLine.discretize(Number=10)

BladeRefShroudSsLine = SolidBladeRef2.Faces[3].Edges[3]
BladeRefShroudSsList = BladeRefShroudSsLine.discretize(Number=10)

BladeRefPoints = BladeRefAvePsList+BladeRefAveSsList+BladeRefHubPsList+BladeRefHubSsList+BladeRefShroudPsList+BladeRefShroudSsList


In [5]:
InitialParam = np.array([[25.0, 30.0, 35.0, 40.0, 0.5, 3.0, 2.0, 0.5, 50], 
                         [30.0, 35.0, 45.0, 50.0, 0.5, 3.0, 2.0, 0.5, 45],
                         [35.0, 50.0, 60.0, 65.0, 0.5, 3.0, 2.0, 0.5, 30]])
InitialParam = InitialParam.flatten()

def CalculationOfDistance(ParamArr):
    
   # Generate profiles
    Beta_arr1 = np.array([ParamArr[0], ParamArr[1], ParamArr[2], ParamArr[3]])
    R_arr1 = np.array([df_shroud['Z'], df_shroud['X']])
    T_arr1 = np.array([ParamArr[4], ParamArr[5], ParamArr[6], ParamArr[7]])

    Beta_arr2 = np.array([ParamArr[9], ParamArr[10], ParamArr[11], ParamArr[12]])
    R_arr2 = np.array([df_ave['Z'], df_ave['X']])
    T_arr2 = np.array([ParamArr[13], ParamArr[14], ParamArr[15], ParamArr[16]])

    Beta_arr3 = np.array([ParamArr[18], ParamArr[19], ParamArr[20], ParamArr[21]])
    R_arr3 = np.array([df_hub['Z'], df_hub['X']])
    T_arr3 = np.array([ParamArr[22], ParamArr[23], ParamArr[24], ParamArr[25]])

    Profile1 = BladeProfile(ParamArr[8], Beta_array=Beta_arr1, R_array=R_arr1, T_array=T_arr1, CoefLe=2, CoefTe=1)
    Profile2 = BladeProfile(ParamArr[17], Beta_array=Beta_arr2, R_array=R_arr2, T_array=T_arr2, CoefLe=2, CoefTe=1)
    Profile3 = BladeProfile(ParamArr[26], Beta_array=Beta_arr3, R_array=R_arr3, T_array=T_arr3, CoefLe=2, CoefTe=1)

    X1, Y1 = Profile1.Generate2Dprofile()
    x1, y1, z1 = Profile1.Generate3Dprofile(X1, Y1)

    X2, Y2 = Profile2.Generate2Dprofile()
    x2, y2, z2 = Profile2.Generate3Dprofile(X2, Y2)

    X3, Y3 = Profile3.Generate2Dprofile()
    x3, y3, z3 = Profile3.Generate3Dprofile(X3, Y3) 

    W1 = Profile1.ProfileToFreeCAD(x1, y1, z1)
    W2 = Profile2.ProfileToFreeCAD(x2, y2, z2)
    W3 = Profile3.ProfileToFreeCAD(x3, y3, z3)

    BladeS = Part.makeLoft([W1, W2, W3])
        #ShroudFace = Part.makeFilledFace([W1.Edges[0], W1.Edges[1], W1.Edges[2], W1.Edges[3]])
        #HubFace = Part.makeFilledFace([W3.Edges[0], W3.Edges[1], W3.Edges[2], W3.Edges[3]])
        #Shell = Part.makeShell([ShroudFace, HubFace, BladeS.Faces[0], BladeS.Faces[1], BladeS.Faces[2], BladeS.Faces[3]])
        #Solid = Part.makeSolid(Shell)
    Part.show(BladeS)

        # Creation of points distance for optimization of the blade shape
    BladeNewAvePsLine = W2.Edges[1]
    BladeNewAvePsList = BladeNewAvePsLine.discretize(Number=10)

    BladeNewAveSsLine = W2.Edges[3]
    BladeNewAveSsList = BladeNewAveSsLine.discretize(Number=10)

    BladeNewHubPsLine = W3.Edges[1]
    BladeNewHubPsList = BladeNewHubPsLine.discretize(Number=10)

    BladeNewHubSsLine = W3.Edges[3]
    BladeNewHubSsList = BladeNewHubSsLine.discretize(Number=10)

    BladeNewShroudPsLine = W1.Edges[1]
    BladeNewShroudPsList = BladeNewShroudPsLine.discretize(Number=10)

    BladeNewShroudSsLine = W1.Edges[3]
    BladeNewShroudSsList = BladeNewShroudSsLine.discretize(Number=10)

    BladeNewPoints = BladeNewAvePsList+BladeNewAveSsList+BladeNewHubPsList+BladeNewHubSsList+BladeNewShroudPsList+BladeNewShroudSsList

    DistanceList = []
    for i in range(len(BladeRefPoints)):
        Distance = BladeRefPoints[i].distanceToPoint(BladeNewPoints[i])
        DistanceList.append(Distance)

    DistanceArray = np.array(DistanceList)
    DistanceAll = DistanceArray.sum()

    f = open('./Distance.txt', 'a')
    f.write(str(DistanceAll)+'\n')
    f.close()

    f2 = open('./Params.txt', 'a')
    for index in ParamArr.tolist():
        f2.write(str(index)+',')
    f2.write('\n')
    f2.close()

    print (DistanceAll)
    FreeCAD.ActiveDocument.saveAs('./Profile.FCStd')
    FreeCAD.ActiveDocument.removeObject('Shape001')
    
    return DistanceAll

In [ ]:
# Optimisation of the distance
bnds = [(20, 40), (25, 55), (30, 65), (40, 70), (0.25, 0.8), (1, 4), (1, 4), (0.25, 1.0), (27, 53)]*3
evo_min = differential_evolution(CalculationOfDistance, bnds, popsize=30, maxiter=200)

291.3130161080043
259.5501392701132
323.1214020427008
186.69074704062086
213.88199183751198
237.8799029386652
367.0407101249086
310.2943620031041
258.50445699296165
358.796156880845
243.67980920623614
322.8462344126189
240.00412331027562
356.79386169689207
193.87130607432695
221.65494902973145
392.02246926252417
263.6136460588108
231.60773344128705
246.6080362931708
456.7623434024268
340.5765170001981
297.6139289976617
244.56526278919682
209.36613128272498
262.20338014771147
244.60721698768486
350.3600630734773
295.01096979132427
361.2378408613589
318.02679749760455
188.82009389206178
425.92634496399216
321.4203886201851
287.8924981687053
298.45009846599436
239.41116179905853
215.98544423293566
254.76038465695103
321.9853217034964
389.29384980962396
264.15490087315965
493.0639858434118
174.47151565766256
220.1638830332493
201.14293576533
327.5943590470022
235.00123462930125
361.06847567208774
378.8504576824097
287.91134056554455
271.1824337178447
258.90517579955326
272.4928606995138
32

387.515935656291
308.36627120517807
252.29182647011095
236.65429280225197
281.43999468751826
234.27028256782293
224.03151103209663
252.5982051142536
267.3172084555863
261.16337264479654
304.7652067674865
332.0487792264149
186.13137688191495
268.7745400156028
237.54469165053717
304.06844204722796
298.8857791537293
242.69291689873936
208.46577697189392
243.86133660173252
320.6383928555506
226.78943158444605
273.24248814827484
258.6753228214781
280.6316138228463
249.97023697701945
258.9205333776663
299.6241456811583
277.5452412687297
274.60328874769573
206.68211905235046
452.68627452626987
332.49685098953375
245.80097859325363
383.47663456286796
208.35999649435632
246.0886351616902
235.7917159551921
249.8012266480639
349.235248323317
380.2847311936185
274.6386068177676
280.68379505822026
272.1078725742297
177.83050569600184
231.19736918470247
298.4912584873832
349.13075610458617
250.00723697113816
256.1075773911063
321.6988994376474
218.8946900402251
215.20719458294585
345.33258693170853


234.5718770370079
226.54041158264377
325.1099241651305
226.93007521945023
296.82595936930977
284.57269496432434
279.99689370969173
259.8820346348783
310.7584991262682
192.37591668550186
336.9156366602145
246.5924834664819
171.3074098960348
240.93630977542597
257.6549059324531
432.101250293534
455.15641141251206
229.53635948074393
221.0330317621086
196.86469745925743
132.4855838013956
207.02381838791968
182.27821194651696
247.1675490554436
168.06854224542593
161.36902632223578
302.9455626077702
205.70517079830628
229.30030926732127
262.44706726045416
385.30486066280565
349.6884654728757
320.1540605071486
176.00799135593726
260.9876183175113
179.24637335031653
390.8510578440144
198.14487513257654
263.9377957596133
269.37460071334596
316.017367223542
367.1019690346796
237.03060909194372
256.1730213826128
165.05490205793706
369.9041521963123
253.2868022470681
260.74393216291
275.40652983603337
178.46511721722877
314.44436132239474
307.7830531817407
372.58130361457637
187.88918900873387
224

185.09622132340206
246.07089551604093
228.5461744478458
209.0169027894033
220.29439152571322
212.90339191632555
215.06970497918329
172.23058659612153
399.9519937474307
205.37660452073112
207.70208328291187
265.42333430284543
291.5778463246141
288.15526993894457
242.49142444070603
220.04951616474116
297.3919925876193
190.27986490821993
251.34289402110684
328.1397813863746
226.73481871081367
240.61870310028704
323.9175316158068
334.8741835683093
237.58284997427836
291.0349967823731
399.61016598999737
226.8320938069957
231.92077548926918
193.31209999592554
277.82622458048854
337.6150816065504
252.6217491434148
285.4646911480868
290.76740406139385
203.80135530716487
259.86188385156044
274.59620873770155
211.9921519066524
258.98512813635773
221.64132258581765
282.8763582771769
327.10722997585435
379.71893967959954
346.6476150088249
250.85873839285532
323.96626803630255
211.70444342371877
281.1143119550014
230.66234043726857
166.93820740377836
246.18598841951797
301.7322124551489
120.3955627

241.6258479535145
263.01954821770556
247.45176838409876
224.83789641778205
335.17525006795825
172.6423599733674
251.9356290556951
176.7677072700224
254.00136713892005
209.16720563605514
250.98326987971947
223.6759658831385
266.09484245018723
214.72405794759771
265.24775579899216
310.0025940172756
261.18060848268453
169.17749684770934
226.3496675924798
188.70920600782588
187.79608763699625
249.57933291457994
217.40505707088877
229.81726290066229
188.41949834579475
150.46730409097103
218.33253770722618
188.43864137502158
212.49362301751592
231.23109739720658
231.665998881413
405.9230431544757
276.8555481501277
201.47718993827615
195.43865517491798
261.90786745056
191.57271751862888
169.18933907750952
163.955518641845
253.47857007312473
160.7606381234993
246.61266615855564
255.04735987517032
351.4750274905992
192.73224612405636
307.9118908604261
256.13322928711364
214.84917278738985
190.51219176174283
178.26136042277065
236.620156957893
279.7596528113908
325.2834390604432
230.652923064215

...

71.21233268443305
55.95482070976636
63.12252234363984
50.978047885220796
165.54836243131925
86.86010813299936
54.37196359917176
58.03140357958941
49.65490029848299
62.22882841339014
54.282468083079365
90.98607150720733
60.17089420259865
72.34472526316375
61.71267312446365
96.55888243089393
204.1785640768969
247.31022811421573
83.52015216695777
149.8510012455084
79.57170162703797
63.524325550565116
58.87569751346842
167.59644512338767
48.58089814508055
60.03019070591094
59.54311854552575
77.04107548838573
52.67365793226213
97.93293148251807
60.19267568177983
54.36758612883502
57.0528875698238
52.86093010493277
62.01995773493843
170.81191825902894
53.745219141734914
208.07366710075863
130.37282473434414
71.28805788736769
163.95314017103985
62.409397135240944
230.17727228392013
52.24162437723962
128.4839832663529
63.37418702147034
63.741988575027925
56.691150600466216
55.657188481342494
57.69512035182018
138.2223307038554
102.8331063851492
198.56394947858666
49.219365460121004
66.40349967

83.81961109055871
69.52916365234456
60.83644588180991
64.9455438714894
83.38657949614829
69.97564025519107
58.147891904489285
181.751717563777
143.3835705711187
55.37408763159532
155.87700747610614
101.73994255806731
82.47301139724347
70.93927116929247
60.68606123316179
71.52999596603591
66.37661124394735
67.25681892823204
62.097899409782826
64.27158512325742
186.799275018385
149.39174784904066
77.72265696167487
54.51570956196394
56.71897855945179
70.95354295526255
63.91703096210756
70.0938536238983
56.108713164936425
78.04419301644731
71.02078307222676
165.2319380261248
85.85601225548236
57.56926951377792
127.33982333455765
55.83754335403137
82.96641720455035
73.14188042842902
50.856358257540485
52.6237136537846
48.86087879151566
47.08436726103915
69.73533842314073
51.30791631365002
60.26535823965826
68.22772998323019
152.55643463111176
51.71598864135684
52.8457199195734
76.48781248662137
53.20056602496389
171.25955949012905
55.803487389776926
100.19560761973304
46.25493289779821
156.

148.61955080283292
71.45992628815057
55.0607868349506
50.18721834616307
93.2658905712871
98.39406552780932
63.04448704184584
51.78815077498781
110.90613049673584
86.49218677783382
50.223868408485124
64.73685140225219
67.07072927231764
58.51382908035792
84.02136961044417
59.47504233104102
71.00566272577557
53.9180570746785
245.23653762054005
89.8381914312343
134.45617517474446
171.0431994450934
51.60423430091225
77.20653332824779
50.99780074647976
66.41104299239737
50.394843778444965
151.82040818499388
82.46007960020465
53.57979537863085
63.741237939145684
61.748239093687765
132.8659925877175
75.37504497423356
52.599765260132294
54.27298997315745
68.97943960571749
70.45289378505377
164.19260193863101
49.594496099141125
94.27942106193959
192.55533324800484
57.59411319272511
53.02020981230531
51.52808675647994
59.36874719888609
56.00168916234086
165.82844578020843
77.16461690171786
162.86861083407805
57.512338515255465
50.3974003659083
221.6470220304211
85.50330334498246
59.85156668366444

50.76601602699773
59.62922941980846
54.56433732739694
50.706554902203244
48.339696213240984
48.784904162755815
55.29790908052473
76.39594556186324
205.68560322561316
140.46026337528565
62.2308767406532
84.52408868486178
51.35934536042087
51.97052533952728
183.9788876899819
86.00064623202715
193.33096669097708
83.49052465959853
47.65737500765656
66.35903603749821
70.07903075920224
54.46231808700618
136.04005672524093
53.308058470249
48.35031229633628
61.037613745945386
66.24848701571042
54.09894594884832
109.88690649961562
53.717418682387844
58.88294034838413
159.4987928164929
53.64742659827087
91.67105151964869
216.94984510254852
51.74517546502334
73.61707860764439
61.15654088287668
50.86921399510096
83.79999749375091
70.460943134396
62.850220684287486
56.59676707798036
73.47939224225571
76.84673847058721
49.132594520152715
80.11420366572982
140.07746472121408
81.1121867372782
54.66620565960635
69.85366078475725
165.7148457346837
155.65224170194836
163.5782098615813
85.54058744819943
7

54.80514433741318
84.52482151379475
47.160693261941745
52.09508325970127
57.477540193363225
224.84247701691007
119.2997409943962
47.404501527460674
53.28475417179255
107.2550200115097
48.76421126140329


In [ ]:
bnds = [(40, 70), (0, 30), (5, 40), (5, 45), (0.1, 0.9), (0.1, 0.9), (0.3, 0.9), (1, 6), (1, 6), (0.1, 0.5), (0.1, 0.9)]*3
bnds.append((35, 75))
res = minimize(func, initial_params, method='SLSQP', bounds=bnds, options={'maxiter': 100, 'disp': True})

In [7]:
CalculationOfDistance(InitialParam)

103.88566458291388


103.88566458291388